# INDEXING AND SELECTING DATA

pandas documentation Indexing 파트의 첫마디.
>The axis labeling information in pandas objects serves many purposes:
- Identifies data (i.e. provides metadata) using known indicators, important for analysis, visualization, and interactive console display. **데이터 구조**를 알려줌
- Enables automatic and explicit data alignment. **피벗테이블** 역할을 함
- Allows intuitive getting and setting of subsets of the data set. **잘라내기** 편함

In this section, we will focus on the final point: namely, how to slice, dice, and generally get and set subsets of pandas objects. The primary focus will be on Series and DataFrame as they have received more development attention in this area.

## 1.Different choices for indexing :`.loc[]`, `.iloc[]`

Getting values from an object with multi-axes selection uses the following notation (using .loc as an example, but the following applies to .iloc as well). Any of the axes accessors may be the **null slice** `:`. Axes left out of the specification are assumed to be :, e.g. p.loc['a'] is equivalent to p.loc['a', :, :].

object type  // indexers

`Series`  //   `s.loc[indexer]`


`DataFrame` // `df.loc[row_indexer, column_indexer]`

### Selection by label: 

`.loc` is primarily label based, but may also be used with a boolean array. .loc will raise KeyError when the items are not found. 

Allowed inputs are:

- single label `.`
- list or array of labels `[]`
- slice object with labels `['a':'f']`
- boolean array, 
- callable function with one argument `[lambda]`



### Selection by Position: 
`.iloc` is primarily integer position based (from 0 to length-1 of the axis), but may also be used with a boolean array.

.iloc will raise IndexError if a requested indexer is out-of-bounds, except slice indexers which allow out-of-bounds indexing. (this conforms with Python/NumPy slice semantics).

Allowed inputs are:

- an integer
- a list or array of integers
- a slice object with ints 1:7
- boolean array
- `callable` function lambda


.loc, .iloc, and also [] indexing can accept a callable as indexer. See more at Selection By Callable.

---
## 2.Basics : `__getitem__` **Lower-dimensional slices**

As mentioned when introducing the data structures in the last section, the primary function of indexing with `[]` (a.k.a. `__getitem__` for those familiar with implementing class behavior in Python) is selecting out **lower-dimensional** slices. The following table shows return type values when indexing pandas objects with []:

Object//Selection//Return Value Type

`Series`//`series[label]`//scalar value

`DataFrame`//`df[col_name]`//`Series` corresponding to colname

In [1]:
import numpy as np
import pandas as pd

In [2]:
dates = pd.date_range('1/1/2020', periods=8)
df = pd.DataFrame(np.random.randn(8, 4), 
                  index=dates, columns=list('ABCD'))


In [3]:
df

,A,B,C,D
2020-01-01,1.262505,1.102292,-0.263778,-0.902797
2020-01-02,0.327848,0.343003,0.346972,0.937495
2020-01-03,0.885922,0.250009,-0.322214,0.360884
2020-01-04,1.991836,-1.145018,0.244568,-0.208436
2020-01-05,0.229071,1.070882,0.806575,1.474958
2020-01-06,-0.759164,-2.356352,0.240075,-1.074884
2020-01-07,-0.322235,-1.166790,-0.914659,0.043936
2020-01-08,-1.374457,-0.533753,-0.856213,-0.887030


In [4]:
s = df['A']

In [5]:
s

2020-01-01    1.262505
2020-01-02    0.327848
2020-01-03    0.885922
2020-01-04    1.991836
2020-01-05    0.229071
2020-01-06   -0.759164
2020-01-07   -0.322235
2020-01-08   -1.374457
Freq: D, Name: A, dtype: float64

In [6]:
s[dates[5]]

-0.7591642052569175

You can pass **a list of columns** to [] to select columns in that order. 

If a column is not contained in the DataFrame, an exception will be raised. 

Multiple columns can also be set in this manner:

In [7]:
df_swap= df.copy()
df_swap[  ['A', 'B']  ] = ['left', 'right']

In [8]:
df_swap

,A,B,C,D
2020-01-01,left,right,-0.263778,-0.902797
2020-01-02,left,right,0.346972,0.937495
2020-01-03,left,right,-0.322214,0.360884
2020-01-04,left,right,0.244568,-0.208436
2020-01-05,left,right,0.806575,1.474958
2020-01-06,left,right,0.240075,-1.074884
2020-01-07,left,right,-0.914659,0.043936
2020-01-08,left,right,-0.856213,-0.887030


In [9]:
df_swap[   ['B', 'A']   ] = df_swap[  ['A', 'B']  ]

In [10]:
df_swap

,A,B,C,D
2020-01-01,right,left,-0.263778,-0.902797
2020-01-02,right,left,0.346972,0.937495
2020-01-03,right,left,-0.322214,0.360884
2020-01-04,right,left,0.244568,-0.208436
2020-01-05,right,left,0.806575,1.474958
2020-01-06,right,left,0.240075,-1.074884
2020-01-07,right,left,-0.914659,0.043936
2020-01-08,right,left,-0.856213,-0.887030


>**Warning** pandas aligns all AXES when setting Series and DataFrame from .loc, and .iloc.
This will not modify df because the **column alignment is before value assignment.**

In [11]:
df_swap= df.copy()
df_swap[  ['A', 'B']  ] = ['left', 'right']

In [12]:
df_swap[  ['A', 'B']  ]

,A,B
2020-01-01,left,right
2020-01-02,left,right
2020-01-03,left,right
2020-01-04,left,right
2020-01-05,left,right
2020-01-06,left,right
2020-01-07,left,right
2020-01-08,left,right


In [13]:
df_swap.loc[:,  ['B', 'A']]  =  df_swap[  ['A', 'B']]  # column alignment (LHS도 df, RHS도 df.....LHS의 칼럼'A'와, RHS의 칼럼 'A' 맞춘 다음 Set value... )
df_swap[  ['A',  'B']  ]

,A,B
2020-01-01,left,right
2020-01-02,left,right
2020-01-03,left,right
2020-01-04,left,right
2020-01-05,left,right
2020-01-06,left,right
2020-01-07,left,right
2020-01-08,left,right


The correct way to swap column values is by using **raw values**

In [14]:
df_swap.loc[:, ['B', 'A'] ] = df_swap[ ['A','B'] ].to_numpy()  # df vs df로 set하지 말고.... df vs raw vlaue로 set하는 습관!
df_swap[ ['A', 'B']]

,A,B
2020-01-01,right,left
2020-01-02,right,left
2020-01-03,right,left
2020-01-04,right,left
2020-01-05,right,left
2020-01-06,right,left
2020-01-07,right,left
2020-01-08,right,left


---
## 3.Attribute access: `.colname`

---
## 4.Slicing ranges with `[]` operator

The most robust and consistent way of slicing ranges along arbitrary axes is described in the Selection by Position section detailing the .iloc method. For now, we explain the semantics of slicing using the [] operator.

With Series, the syntax works exactly as with an ndarray, returning a slice of the values and the corresponding labels:

In [15]:
s[:5]

2020-01-01    1.262505
2020-01-02    0.327848
2020-01-03    0.885922
2020-01-04    1.991836
2020-01-05    0.229071
Freq: D, Name: A, dtype: float64

In [16]:
s[::2]

2020-01-01    1.262505
2020-01-03    0.885922
2020-01-05    0.229071
2020-01-07   -0.322235
Freq: 2D, Name: A, dtype: float64

In [17]:
s[::-1]

2020-01-08   -1.374457
2020-01-07   -0.322235
2020-01-06   -0.759164
2020-01-05    0.229071
2020-01-04    1.991836
2020-01-03    0.885922
2020-01-02    0.327848
2020-01-01    1.262505
Freq: -1D, Name: A, dtype: float64

Note that setting works as well:

In [18]:
s2 = s.copy()

In [19]:
s2[:5] = 0
s2

2020-01-01    0.000000
2020-01-02    0.000000
2020-01-03    0.000000
2020-01-04    0.000000
2020-01-05    0.000000
2020-01-06   -0.759164
2020-01-07   -0.322235
2020-01-08   -1.374457
Freq: D, Name: A, dtype: float64

With DataFrame, slicing inside of `[]` **slices the rows**. This is provided largely as a convenience since it is such a common operation.

In [20]:
df[:3]

,A,B,C,D
2020-01-01,1.262505,1.102292,-0.263778,-0.902797
2020-01-02,0.327848,0.343003,0.346972,0.937495
2020-01-03,0.885922,0.250009,-0.322214,0.360884


In [21]:
df[::-1]

,A,B,C,D
2020-01-08,-1.374457,-0.533753,-0.856213,-0.887030
2020-01-07,-0.322235,-1.166790,-0.914659,0.043936
2020-01-06,-0.759164,-2.356352,0.240075,-1.074884
2020-01-05,0.229071,1.070882,0.806575,1.474958
2020-01-04,1.991836,-1.145018,0.244568,-0.208436
2020-01-03,0.885922,0.250009,-0.322214,0.360884
2020-01-02,0.327848,0.343003,0.346972,0.937495
2020-01-01,1.262505,1.102292,-0.263778,-0.902797


---
## 5.Selection by label: `.loc`

pandas는 purely label based indexing을 지향한다. 이러한 인덱싱 기법은 엄격한 기준을 따라 작동하게 됨. 
- 인덱싱하는 모든 label은 데이터의 index에 포함되어 있어야 하며, 그렇지 않은 경우 KeyError를 띄운다. 
- 슬라이싱의 경우, 시작점과 종료지점은 모두 included 이다. 
- Integer 형식도 valid labels이지만, 이는 position base가 아님을 유념해야 한다.

.loc 을 통해 엑셀과 유사한 indexing을 할 수 있다. 
 * a single label 'a'
 * a list or array of labels ['a','b','c']
 * a slice object with labels 'a':'f'
 * a boolean array
 * a callable

In [22]:
s1 = pd.Series(np.random.randn(6), index=list('abcdef'))

In [23]:
s1

a    0.936014
b    1.128088
c    1.172766
d   -0.932735
e   -2.307960
f    0.206326
dtype: float64

In [24]:
s1.loc['b']

1.1280884925618901

In [25]:
s1.loc['c':]

c    1.172766
d   -0.932735
e   -2.307960
f    0.206326
dtype: float64

Note that setting works as well:

In [26]:
s1.loc['c':] = 0
s1

a    0.936014
b    1.128088
c    0.000000
d    0.000000
e    0.000000
f    0.000000
dtype: float64

With a DataFrame:

In [27]:
df1 = pd.DataFrame(np.random.randn(6,4),
                  index=list('abcdef'),
                  columns=list('ABCD'))

In [28]:
df1

,A,B,C,D
a,1.799367,0.523165,-0.477684,1.781191
b,0.447059,0.271733,-0.803050,-0.560528
c,0.581600,0.958971,1.186200,-1.075818
d,0.279810,-2.407943,-2.257287,0.161710
e,-1.951617,0.203779,1.169619,0.179838
f,-0.412401,0.225526,-1.081073,-0.946060


In [29]:
df1.loc[['a','c','d']]

,A,B,C,D
a,1.799367,0.523165,-0.477684,1.781191
c,0.581600,0.958971,1.186200,-1.075818
d,0.279810,-2.407943,-2.257287,0.161710


In [30]:
df1.loc[['a','b']]

,A,B,C,D
a,1.799367,0.523165,-0.477684,1.781191
b,0.447059,0.271733,-0.803050,-0.560528


In [31]:
df1.loc[['a','b'], :]  #explicit column is better

,A,B,C,D
a,1.799367,0.523165,-0.477684,1.781191
b,0.447059,0.271733,-0.803050,-0.560528


Accessing via label slices:

In [32]:
df1.loc['d':, 'A':'C']

,A,B,C
d,0.279810,-2.407943,-2.257287
e,-1.951617,0.203779,1.169619
f,-0.412401,0.225526,-1.081073


For getting a cross section(reduce to a Series) using a label

In [33]:
df1.loc['a']

A    1.799367
B    0.523165
C   -0.477684
D    1.781191
Name: a, dtype: float64

For getting values with a boolean array:

In [34]:
df1.loc['a']>0

A     True
B     True
C    False
D     True
Name: a, dtype: bool

In [35]:
df1.loc[:, df1.loc['a'] > 0]

,A,B,D
a,1.799367,0.523165,1.781191
b,0.447059,0.271733,-0.560528
c,0.581600,0.958971,-1.075818
d,0.279810,-2.407943,0.161710
e,-1.951617,0.203779,0.179838
f,-0.412401,0.225526,-0.946060


For getting a value explicitly..(equiv to deprecated df.get_value('a','A')

In [36]:
df1.loc['a','A']

1.7993671839845506

#### Slicing With Labels: `.loc[row_indexer, col_indexer]`
When using .loc with slices, if both the start and the stop labels are present in the index, then elements _located_ between the two (including them) are returned.

In [37]:
s = pd.Series(list('abcde'), index=[0,3,2,5,4])

In [38]:
s

0    a
3    b
2    c
5    d
4    e
dtype: object

In [39]:
s.loc[3:4]

3    b
2    c
5    d
4    e
dtype: object

If at least one of the two is absent, but the index is **sorted**, and can be compared against start and stop labels, then slicing will still work as expected, by selecting labels which _rank_ between the two:

In [40]:
s.sort_index()

0    a
2    c
3    b
4    e
5    d
dtype: object

In [41]:
s  # s는 그대로 있음

0    a
3    b
2    c
5    d
4    e
dtype: object

In [42]:
s.sort_index().loc[1:6]

2    c
3    b
4    e
5    d
dtype: object

However, if at least one of the two is absent and the index is not sorted, an error will be raised (since doing otherwise would be computationally expensive, as well as potentially ambiguous for mixed type indexes). For instance, in the above example, s.loc[1:6] would raise KeyError.

For the rationale behind this behavior, see [Endpoints are inclusive](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced-endpoints-are-inclusive).

---
## 6.Selection by position

pandas는 Integer-based indexing도 지원함(엑셀의 참조 방법이 두가지인 것처럼)
Numpy나 Python의 slicing은 Position based indexing을 지향함. 단순하긴 한데, 코드 작성자가 데이터구조를 고려해 Axis를 정해줘야 해서 불편함.
직관적인 label-based indexing(if included..)을 애용합시다.
더욱이나 Integer-based indexing은 start bound is included, while the upper bound is excluded.라는 점을 유념해야 함.


In [43]:
s1 = pd.Series(np.random.randn(5), index=list(range(0, 10, 2)))

In [44]:
s1

0    1.559801
2   -0.372994
4    1.859339
6   -1.251335
8   -1.339381
dtype: float64

In [45]:
s1.iloc[:3]

0    1.559801
2   -0.372994
4    1.859339
dtype: float64

In [46]:
s1.iloc[3]

-1.2513352907142412

Note that setting works as well:

In [47]:
s1.iloc[:3] = 0

In [48]:
s1

0    0.000000
2    0.000000
4    0.000000
6   -1.251335
8   -1.339381
dtype: float64

With a DataFrame:

In [49]:
df1 = pd.DataFrame(np.random.randn(6,4),
                  index=list(range(0, 12, 2)),
                  columns=list(range(0, 8, 2)))


In [50]:
df1

,0,2,4,6
0,-1.111377,-0.497802,-0.284241,2.121045
2,0.203534,0.436939,-1.183794,0.137551
4,-0.653486,-0.069763,-0.838719,0.488276
6,-0.199757,-0.029538,0.606245,-1.138398
8,1.595537,-0.116848,-0.759880,2.367896
10,0.148621,-1.276391,1.139486,-1.370369


Select via integer slicing:

In [51]:
df1.iloc[:3]

,0,2,4,6
0,-1.111377,-0.497802,-0.284241,2.121045
2,0.203534,0.436939,-1.183794,0.137551
4,-0.653486,-0.069763,-0.838719,0.488276


In [52]:
df1.iloc[1:5, 2:4]

,4,6
2,-1.183794,0.137551
4,-0.838719,0.488276
6,0.606245,-1.138398
8,-0.759880,2.367896


Select via integer list:

In [53]:
df1.iloc[[1,3,5], [1,3]]

,2,6
2,0.436939,0.137551
6,-0.029538,-1.138398
10,-1.276391,-1.370369


In [54]:
df1.iloc[1:3, :] #1포지션부터 +2개 행, 모든열

,0,2,4,6
2,0.203534,0.436939,-1.183794,0.137551
4,-0.653486,-0.069763,-0.838719,0.488276


In [55]:
df1.iloc[:, 1:3] #모든 행, 1포지션부터 2개열

,2,4
0,-0.497802,-0.284241
2,0.436939,-1.183794
4,-0.069763,-0.838719
6,-0.029538,0.606245
8,-0.116848,-0.759880
10,-1.276391,1.139486


In [56]:
df1.iloc[1]  #1포지션 행

0    0.203534
2    0.436939
4   -1.183794
6    0.137551
Name: 2, dtype: float64

Out of range slice indexes are handled gracefully just as in Python/Numpy

In [57]:
x = list('abcdef')

In [58]:
x

['a', 'b', 'c', 'd', 'e', 'f']

In [59]:
x[4:10]  #포지션4('e')부터 길이는 최대 2개이므로 4:6이 엄밀하게 맞는 코드이나, 6개를 불러오라 해도 알아서 끝까지만 탐색

['e', 'f']

In [60]:
x[8:10]

[]

In [61]:
s=pd.Series(x)

In [62]:
s

0    a
1    b
2    c
3    d
4    e
5    f
dtype: object

In [63]:
s.iloc[4:10]

4    e
5    f
dtype: object

In [64]:
s.iloc[8:10]

Series([], dtype: object)

Note that using slices that go out of bounds can result in an empty axis.(an empty Df being returned)

In [65]:
dfl = pd.DataFrame(np.random.randn(5, 2), columns=list('AB'))

In [66]:
dfl

,A,B
0,0.204219,0.169506
1,-0.697223,1.356001
2,1.170873,1.424745
3,0.762349,1.227938
4,-0.023275,1.171791


In [67]:
dfl.iloc[:, 2:3] #모든 행, 2포지션에서 1개열 뽑아와.... 근데 2포지션? 없는데? 열이 2개니까 1포지션까지만 있지(0,1)

""
0
1
2
3
4


In [68]:
dfl.iloc[:, 1:3]

,B
0,0.169506
1,1.356001
2,1.424745
3,1.227938
4,1.171791


In [69]:
dfl.iloc[4:6] #4포 행에서 2개 행 뽑아와... 근데 행개수 5개인데...

,A,B
4,-0.023275,1.171791


A single indexer that is out of bounds will raise an `IndexError`. A list of indexers where any element is out of bounds will raise an `indexerror`

In [70]:
dfl.iloc[5]

IndexError: single positional indexer is out-of-bounds

In [ ]:
dfl.iloc[[4,5,6]]

In [71]:
dfl.iloc[:, 4]

IndexError: single positional indexer is out-of-bounds

---
## 8. Selection by callable
.loc, .iloc, and also [] indexing can accept a **callable as indexer**. The callable must be a function with one argument(the calling Series or DataFrame) that returns valid output for indexing.

In [ ]:
df1 = pd.DataFrame(np.random.randn(6,4),
                  index=list('abcdef'),
                  columns=list('ABCD'))
df1

In [ ]:
df1.loc[lambda df: df.A > 0, :] #행은 콜러블...콜링 df1 자기자신...A열... 0보다 큰 행...., 모든 열을 불러와...

In [72]:
df1.loc[df1.A>0, :] #equivalent Selection by boolean array

AttributeError: 'DataFrame' object has no attribute 'A'

In [ ]:
df1.loc[:, lambda df: ['A', 'B']]

In [ ]:
df1.iloc[:, lambda df: [0,1]]

In [ ]:
df1[lambda df: df.columns[0]] # callable return... 'A'

Use callable indexing in Series

In [73]:
df1.A.loc[lambda s: s> 0]

AttributeError: 'DataFrame' object has no attribute 'A'

Using these methods.. you can chain data selection operations without using a temporary variable...**AMAZING!!!**

In [ ]:
bb= pd.read_csv('data/baseball.csv', index_col='id')

In [74]:
(bb.groupby(['year', 'team'])).sum().loc[lambda df: df.r > 100]

NameError: name 'bb' is not defined

---
## 9. IX indexer is deprecated

---
## 10. Indexing with list with missing labels is deprecated

Indexing with list with missing labels is deprecated, in favor of `.reindex`

In [75]:
s=pd.Series([1,2,3])

In [76]:
s

0    1
1    2
2    3
dtype: int64

In [77]:
s.loc[[1,2]]

1    2
2    3
dtype: int64

In [78]:
s.loc[[1,2,3]]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


1    2.0
2    3.0
3    NaN
dtype: float64

### Reindexing.. 
the idiomatic way to achieve selecting potentially not-found elements.

See also [reindexing_basic functionality](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html#basics-reindexing)

In [79]:
s.reindex([1,2,3])

1    2.0
2    3.0
3    NaN
dtype: float64

In [80]:
labels=[1,2,3]

Alternatively, selecting only valid keys..... it is guaranteed to preserve the dtype of the selection

In [81]:
s.loc[s.index.intersection(labels)]

1    2
2    3
dtype: int64

In [82]:
s = pd.Series(np.arange(4), index=['a','a','b','c'])

In [83]:
s

a    0
a    1
b    2
c    3
dtype: int32

In [84]:
labels = ['c','d']

In [85]:
s.reindex(labels)

ValueError: cannot reindex from a duplicate axis

Duplicate axis인 경우..... intersection으로 중복된 axis를 풀어주고... .reindex 하면 됨

In [86]:
s.loc[s.index.intersection(labels)].reindex(labels)

c    3.0
d    NaN
dtype: float64

In [87]:
labels = ['a','d']

In [88]:
s.loc[s.index.intersection(labels)]

a    0
a    1
a    0
a    1
dtype: int32

In [89]:
s.loc[s.index.intersection(labels)].reindex(labels)

ValueError: cannot reindex from a duplicate axis

---
## 11. Selecting random samples

A random selection of rows or columns from a Series or DataFrame with the sample() method. The method will sample rows by default, and accepts a specific number of rows/columns to return, or a fraction of rows.

In [90]:
s=pd.Series([0,1,2,3,4,5])

In [91]:
s.sample()

2    2
dtype: int64

In [92]:
s.sample(n=3)

0    0
2    2
3    3
dtype: int64

In [93]:
s.sample(frac=0.5)

2    2
0    0
4    4
dtype: int64

By default, sample will return each row at most once, but one can also sample with replacement using the replace option:

In [94]:
s.sample(n=6, replace=True)

4    4
4    4
5    5
4    4
5    5
5    5
dtype: int64

By default, each row has an equal probability of being selected, but if you want rows to have different probabilities, you can pass the sample function sampling weights as weights. These weights can be a list, a NumPy array, or a Series, but they must be of the same length as the object you are sampling. Missing values will be treated as a weight of zero, and inf values are not allowed. If weights do not sum to 1, they will be re-normalized by dividing all weights by the sum of the weights. For example:


In [95]:
ex_weights = pd.Series([0, 0, 0.2, 0.2, 0.2, 0.4], index=s.index, name="prob")

In [96]:
ex_weights

0    0.0
1    0.0
2    0.2
3    0.2
4    0.2
5    0.4
Name: prob, dtype: float64

In [97]:
s.sample(n=3, weights=ex_weights)

4    4
2    2
5    5
dtype: int64

When applied to a DF, you can use a column of the DF as sampling weights (provided you ar sampling rows and not columns) by simply passing the name of the column as string

In [98]:
df2 = pd.DataFrame({'col1': [9,8,7,6],
                    'weight_column': [0.5, 0.4, 0.1, 0]})

In [99]:
df2.sample(n=3, weights='weight_column')

,col1,weight_column
1,8,0.4
0,9,0.5
2,7,0.1


sample also allows users to sample columns instead of rows using the axis argument.

In [100]:
df2.sample(n=1, axis=1)

,col1
0,9
1,8
2,7
3,6


Finally, one can also set a seed for sample’s random number generator using the random_state argument, which will accept either an integer (as a seed) or a NumPy RandomState object.

In [101]:
df4 = pd.DataFrame({'col1': [1,2,3], 'col2': [2,3,4]})

In [102]:
df4.sample(2, random_state=2)

,col1,col2
2,3,4
1,2,3


In [103]:
df4.sample(2, random_state=2)

,col1,col2
2,3,4
1,2,3


---
## 12. Setting with enlargement

The .loc/[] operations can perform enlargement when setting a non-existent key for that axis.

In the Series case this is effectively an **appending operation.**

In [104]:
se = pd.Series([1,2,3])

In [105]:
se

0    1
1    2
2    3
dtype: int64

In [106]:
se[5]=5.

In [107]:
se

0    1.0
1    2.0
2    3.0
5    5.0
dtype: float64

A DataFrame can be **enlarged** on either **axis** via .loc.

In [108]:
dfi = pd.DataFrame(np.arange(6).reshape(3,2),
                  columns=['A','B'])

In [109]:
dfi.loc[:, 'C'] = dfi.loc[:, 'A']  # enlarge column

In [110]:
dfi

,A,B,C
0,0,1,0
1,2,3,2
2,4,5,4


In [111]:
dfi.loc[3] = 5  # appending row

In [112]:
dfi

,A,B,C
0,0,1,0
1,2,3,2
2,4,5,4
3,5,5,5


---
## 13. Fast scalar value getting and setting

Since indexing with `[]` must handle a lot of cases (single-label access, slicing, boolean indexing, etc.), it has a bit of overhead in order to figure out what you’re asking for. 

If you only want to access a scalar value, the **fastest** way is to use the `.at` and `.iat` methods, which are implemented on all of the data structures.

Similarly to loc, `.at` provides label based **scalar lookups**, while, `.iat` provides integer based lookups analogously to iloc

In [113]:
s = pd.Series(range(6))

In [114]:
s

0    0
1    1
2    2
3    3
4    4
5    5
dtype: int64

In [115]:
s.iat[5]

5

You can also set using these same indexers.

In [116]:
dates = pd.date_range('2000-01-01', periods=8)

In [117]:
df = pd.DataFrame(np.random.randn(8,4),
                 index=dates, 
                 columns=list('ABCD'))

In [118]:
df.at[dates[5], 'A']

0.1561988662392702

In [119]:
df.loc[dates[5], 'A']

0.1561988662392702

In [120]:
df.iat[3,0]

-0.12510017395398682

In [121]:
df.at[dates[5], 'E']=7
df.iat[3,0] = 7

In [122]:
df

,A,B,C,D,E
2000-01-01,1.089679,-0.298857,-0.339292,-1.405819,NaN
2000-01-02,-1.365324,0.414619,-0.090180,2.015663,NaN
2000-01-03,2.552220,-0.908557,-0.012032,0.719043,NaN
2000-01-04,7.000000,-0.479034,0.255735,0.980161,NaN
2000-01-05,0.681791,-0.669006,-0.991850,-0.101555,NaN
2000-01-06,0.156199,1.417298,0.560109,-1.240861,7.0
2000-01-07,-0.278914,0.789748,-0.005611,0.867624,NaN
2000-01-08,2.073882,0.208156,0.724563,0.642229,NaN


`at` may **enlarge** the object in-place as above if the indexer is missing.

In [123]:
df.at[dates[-1] + pd.Timedelta('1 day'), 0] = 7

In [124]:
df

,A,B,C,D,E,0
2000-01-01,1.089679,-0.298857,-0.339292,-1.405819,NaN,NaN
2000-01-02,-1.365324,0.414619,-0.090180,2.015663,NaN,NaN
2000-01-03,2.552220,-0.908557,-0.012032,0.719043,NaN,NaN
2000-01-04,7.000000,-0.479034,0.255735,0.980161,NaN,NaN
2000-01-05,0.681791,-0.669006,-0.991850,-0.101555,NaN,NaN
2000-01-06,0.156199,1.417298,0.560109,-1.240861,7.0,NaN
2000-01-07,-0.278914,0.789748,-0.005611,0.867624,NaN,NaN
2000-01-08,2.073882,0.208156,0.724563,0.642229,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN,NaN,7.0


---
## 14. Boolean indexing
Another common operation is the use of boolean vectors to filter the dat. The operators are | for or, & for and , ~ for not.
These __must__ be grouped by using parentheses, since by default Python will evaluate an expression such as df.A >2 & df.B < 3 as df.A > (2& df.B) < 3, while the desired evaluation order is (df.A>2) & (df.B <3)

In [125]:
s = pd.Series(range(-3,4))

In [126]:
range(-3,4)

range(-3, 4)

In [127]:
s

0   -3
1   -2
2   -1
3    0
4    1
5    2
6    3
dtype: int64

In [128]:
s[s>0]

4    1
5    2
6    3
dtype: int64

In [129]:
s[(s<-1)|(s>0.5)]

0   -3
1   -2
4    1
5    2
6    3
dtype: int64

In [130]:
s[~(s<0)]

3    0
4    1
5    2
6    3
dtype: int64

In [131]:
df[df['A'] > 0]

,A,B,C,D,E,0
2000-01-01,1.089679,-0.298857,-0.339292,-1.405819,NaN,NaN
2000-01-03,2.552220,-0.908557,-0.012032,0.719043,NaN,NaN
2000-01-04,7.000000,-0.479034,0.255735,0.980161,NaN,NaN
2000-01-05,0.681791,-0.669006,-0.991850,-0.101555,NaN,NaN
2000-01-06,0.156199,1.417298,0.560109,-1.240861,7.0,NaN
2000-01-08,2.073882,0.208156,0.724563,0.642229,NaN,NaN


In [132]:
df2 = pd.DataFrame({'a': ['one', 'one', 'two', 'three', 'two', 'one', 'six'],
                    'b': ['x', 'y', 'y', 'x', 'y', 'x', 'x'],
                    'c': np.random.randn(7)})

In [133]:
df2

,a,b,c
0,one,x,-1.053556
1,one,y,-0.770077
2,two,y,0.566672
3,three,x,-0.717784
4,two,y,1.047404
5,one,x,-0.207997
6,six,x,-1.234119


List comprehension and the map methosd of series can also be used to produce more complex criteria

In [134]:
criterion = df2['a'].map(lambda x: x.startswith('t'))

In [135]:
criterion

0    False
1    False
2     True
3     True
4     True
5    False
6    False
Name: a, dtype: bool

In [136]:
df2[criterion]

,a,b,c
2,two,y,0.566672
3,three,x,-0.717784
4,two,y,1.047404


In [137]:
#equivalent but slower
df2[[x.startswith('t') for x in df2['a']]]

,a,b,c
2,two,y,0.566672
3,three,x,-0.717784
4,two,y,1.047404


In [138]:
#Multiple criteria
df2[criterion & (df2['b'] == 'x')]

,a,b,c
3,three,x,-0.717784


In [139]:
df2.loc[lambda df: df.a.map(lambda x: x.startswith('t'))]

,a,b,c
2,two,y,0.566672
3,three,x,-0.717784
4,two,y,1.047404


In [140]:
df2.loc[criterion & (df2['b'] == 'x'), 'b':'c']

,b,c
3,x,-0.717784


---
## 15. Indexing with `.isin()`: Membership Check
Consider the `isin()` method of `Series`, which returns a boolean vector that is true wherever the Series' **elements exist** in the passed list. 

This allows you to select rows where one or more columns have values you want 

In [141]:
s = pd.Series(np.arange(5), index=np.arange(5)[::-1], dtype='int64')

In [142]:
s

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [143]:
s.isin([2, 4, 6])

4    False
3    False
2     True
1    False
0     True
dtype: bool

In [144]:
s[s.isin([2, 4, 6])] #lookup value

2    2
0    4
dtype: int64

The same method is available for `Index` objects and is useful for the cases when you don’t know which of the sought labels are in fact present:

In [145]:
s[s.index.isin([2, 4, 6])] #lookup index... 근데 존재하는지 알지는 못할 경우

4    0
2    2
dtype: int64

Compare it to the following: Index sort, Non-existing label, dtype

In [146]:
s.reindex([2, 4, 6])

2    2.0
4    0.0
6    NaN
dtype: float64

In addition to that, MultiIndex allows selecting a separate level to use in the membership check:


In [147]:
s_mi = pd.Series(np.arange(6),
                index=pd.MultiIndex.from_product([[0,1],['a','b','c']]))

In [148]:
s_mi

0  a    0
   b    1
   c    2
1  a    3
   b    4
   c    5
dtype: int32

In [149]:
s_mi.iloc[s_mi.index.isin([(1,'a'), (2,'b'), (0,'c')])]

0  c    2
1  a    3
dtype: int32

In [150]:
s_mi.iloc[s_mi.index.isin(['a','c','e'], level=1)]

0  a    0
   c    2
1  a    3
   c    5
dtype: int32

DataFrame also has an isin() method. When calling isin, pass a set of values as either an `array []` OR `dict {}`. If values is an array, isin returns a DataFrame of booleans that is the same shape as the original DataFrame, with True wherever the element is in the sequence of values.

In [151]:
df = pd.DataFrame({'vals': [1, 2, 3, 4], 'ids': ['a', 'b', 'f', 'n'],
                   'ids2': ['a', 'n', 'c', 'n']})

In [152]:
df

,vals,ids,ids2
0,1,a,a
1,2,b,n
2,3,f,c
3,4,n,n


In [153]:
values=['a','b',1,3]

In [154]:
df.isin(values)

,vals,ids,ids2
0,True,True,True
1,False,True,False
2,True,False,False
3,False,False,False


Oftentimes you’ll want to match certain values with certain columns. Just make values a `dict {key: value}` where the key is the column, and the value is a list of items you want to check for.

In [155]:
values={'ids':['a','b'], 'vals':[1,3]}

In [156]:
df.isin(values)

,vals,ids,ids2
0,True,True,False
1,False,True,False
2,True,False,False
3,False,False,False


Combine DataFrame’s isin with the `any()` and `all()` methods to quickly select **subsets** of your data that meet a given criteria. To select a row where each column meets its own criterion:

In [157]:
values = {'ids': ['a', 'b'], 'ids2': ['a', 'c'], 'vals': [1, 3]}

In [158]:
row_mask = df.isin(values).all(1)

In [159]:
row_mask

0     True
1    False
2    False
3    False
dtype: bool

In [160]:
df.loc[row_mask]

,vals,ids,ids2
0,1,a,a


---
## 16. The `.where()` Method and Masking
Selecting values from a series with a boolean vector generally returns a subset of the data. To guarantee that selection output has the __same shape as the original data__(just as what `isin()` returns), you can use the `where()` method in Series and DataFrame.

In [161]:
s

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [162]:
s[s>0]

3    1
2    2
1    3
0    4
dtype: int64

In [163]:
s.where(s>0)

4    NaN
3    1.0
2    2.0
1    3.0
0    4.0
dtype: float64

In [164]:
s.where(s.isin([1,2,3,4]))

4    NaN
3    1.0
2    2.0
1    3.0
0    4.0
dtype: float64

Selecting values from a DF with a boolean criterion now also preserves input data shape. `where()` is used under the hood as the implementation. the code below is equivalent to df.where(df < 0)

In [165]:
dates = pd.date_range('20010101', periods=8)
df = pd.DataFrame(np.random.randn(8,3),
                 index=dates)

In [166]:
df

,0,1,2
2001-01-01,0.199353,0.958447,-0.890055
2001-01-02,0.520690,-0.271940,-0.949279
2001-01-03,-0.813522,0.005863,-1.872484
2001-01-04,-0.537690,-1.451723,-0.623780
2001-01-05,0.897283,-0.642548,-1.014361
2001-01-06,0.325985,1.749058,-0.988162
2001-01-07,0.664227,0.578281,0.122327
2001-01-08,-1.262073,0.142682,-0.515181


In [167]:
df[df< 0]

,0,1,2
2001-01-01,NaN,NaN,-0.890055
2001-01-02,NaN,-0.271940,-0.949279
2001-01-03,-0.813522,NaN,-1.872484
2001-01-04,-0.537690,-1.451723,-0.623780
2001-01-05,NaN,-0.642548,-1.014361
2001-01-06,NaN,NaN,-0.988162
2001-01-07,NaN,NaN,NaN
2001-01-08,-1.262073,NaN,-0.515181


In [168]:
df.where(df<0)

,0,1,2
2001-01-01,NaN,NaN,-0.890055
2001-01-02,NaN,-0.271940,-0.949279
2001-01-03,-0.813522,NaN,-1.872484
2001-01-04,-0.537690,-1.451723,-0.623780
2001-01-05,NaN,-0.642548,-1.014361
2001-01-06,NaN,NaN,-0.988162
2001-01-07,NaN,NaN,NaN
2001-01-08,-1.262073,NaN,-0.515181


Compared to `[df<0]`, where() can do something else...

In addition, where takes an optional `other` argument for replacement of values where the condition is False, in the returned copy.

In [169]:
df.where(df<0, other=-df)

,0,1,2
2001-01-01,-0.199353,-0.958447,-0.890055
2001-01-02,-0.520690,-0.271940,-0.949279
2001-01-03,-0.813522,-0.005863,-1.872484
2001-01-04,-0.537690,-1.451723,-0.623780
2001-01-05,-0.897283,-0.642548,-1.014361
2001-01-06,-0.325985,-1.749058,-0.988162
2001-01-07,-0.664227,-0.578281,-0.122327
2001-01-08,-1.262073,-0.142682,-0.515181


You may wish to set values based on some boolean criteria. This can be done intuitively like so:

In [170]:
s2=s.copy()
s2

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [171]:
s2[s2<1] = 999

In [172]:
s2

4    999
3      1
2      2
1      3
0      4
dtype: int64

In [173]:
df2 = df.copy()

In [174]:
df2[df2 <0] = 0

In [175]:
df2

,0,1,2
2001-01-01,0.199353,0.958447,0.000000
2001-01-02,0.520690,0.000000,0.000000
2001-01-03,0.000000,0.005863,0.000000
2001-01-04,0.000000,0.000000,0.000000
2001-01-05,0.897283,0.000000,0.000000
2001-01-06,0.325985,1.749058,0.000000
2001-01-07,0.664227,0.578281,0.122327
2001-01-08,0.000000,0.142682,0.000000


By default, where returns a modified copy of the data. There is an optional parameter inplace so that the original data can be modified without creating a copy:

In [176]:
df_orig = df.copy()

In [177]:
df_orig

,0,1,2
2001-01-01,0.199353,0.958447,-0.890055
2001-01-02,0.520690,-0.271940,-0.949279
2001-01-03,-0.813522,0.005863,-1.872484
2001-01-04,-0.537690,-1.451723,-0.623780
2001-01-05,0.897283,-0.642548,-1.014361
2001-01-06,0.325985,1.749058,-0.988162
2001-01-07,0.664227,0.578281,0.122327
2001-01-08,-1.262073,0.142682,-0.515181


In [178]:
df_orig.where(df > 0, other=-df, inplace=True)

In [179]:
df_orig

,0,1,2
2001-01-01,0.199353,0.958447,0.890055
2001-01-02,0.520690,0.271940,0.949279
2001-01-03,0.813522,0.005863,1.872484
2001-01-04,0.537690,1.451723,0.623780
2001-01-05,0.897283,0.642548,1.014361
2001-01-06,0.325985,1.749058,0.988162
2001-01-07,0.664227,0.578281,0.122327
2001-01-08,1.262073,0.142682,0.515181


> **NOTE** The signature for DataFrame.where() differs from numpy.where(). Roughly df1.where(m, df2) is equivalent to np.where(m, df1, df2).

In [180]:
df.where(df < 0, -df) == np.where(df < 0, df, -df)

,0,1,2
2001-01-01,True,True,True
2001-01-02,True,True,True
2001-01-03,True,True,True
2001-01-04,True,True,True
2001-01-05,True,True,True
2001-01-06,True,True,True
2001-01-07,True,True,True
2001-01-08,True,True,True


### Alignment
Furthermorem `where` aligns the input boolean condition (ndarray or DataFrame), such that **partial selection** with setting is possible.
이건 판다스의 Label 지향성 때문에 가능한 기능!

In [181]:
df2 = df.copy()

In [182]:
df2

,0,1,2
2001-01-01,0.199353,0.958447,-0.890055
2001-01-02,0.520690,-0.271940,-0.949279
2001-01-03,-0.813522,0.005863,-1.872484
2001-01-04,-0.537690,-1.451723,-0.623780
2001-01-05,0.897283,-0.642548,-1.014361
2001-01-06,0.325985,1.749058,-0.988162
2001-01-07,0.664227,0.578281,0.122327
2001-01-08,-1.262073,0.142682,-0.515181


In [183]:
df2[1:4]  > 0 

,0,1,2
2001-01-02,True,False,False
2001-01-03,False,True,False
2001-01-04,False,False,False


In [184]:
df2[df2[1:4] > 0] #Align으로 지자리 찾아 박힘

,0,1,2
2001-01-01,NaN,NaN,NaN
2001-01-02,0.52069,NaN,NaN
2001-01-03,NaN,0.005863,NaN
2001-01-04,NaN,NaN,NaN
2001-01-05,NaN,NaN,NaN
2001-01-06,NaN,NaN,NaN
2001-01-07,NaN,NaN,NaN
2001-01-08,NaN,NaN,NaN


In [185]:
df2[df2[1:4] > 0] = 'Fuck'  

In [186]:
df2

,0,1,2
2001-01-01,0.199353,0.958447,-0.890055
2001-01-02,Fuck,-0.27194,-0.949279
2001-01-03,-0.813522,Fuck,-1.872484
2001-01-04,-0.53769,-1.45172,-0.623780
2001-01-05,0.897283,-0.642548,-1.014361
2001-01-06,0.325985,1.74906,-0.988162
2001-01-07,0.664227,0.578281,0.122327
2001-01-08,-1.26207,0.142682,-0.515181


`where` can also accept axis and level parameters to align the input when performing the where

In [187]:
df.columns = ['A','B','C']

In [188]:
df2 = df.copy()

In [189]:
cond = df2>0

In [190]:
df2.where(cond)

,A,B,C
2001-01-01,0.199353,0.958447,NaN
2001-01-02,0.520690,NaN,NaN
2001-01-03,NaN,0.005863,NaN
2001-01-04,NaN,NaN,NaN
2001-01-05,0.897283,NaN,NaN
2001-01-06,0.325985,1.749058,NaN
2001-01-07,0.664227,0.578281,0.122327
2001-01-08,NaN,0.142682,NaN


In [191]:
df2.where(cond, other=df2['A'], axis = 'index') #other = df2['A']인데 index방향으로 끌어다 쓸지, column 방향으로 끌어다 쓸지 알려줘야지..!

,A,B,C
2001-01-01,0.199353,0.958447,0.199353
2001-01-02,0.520690,0.520690,0.520690
2001-01-03,-0.813522,0.005863,-0.813522
2001-01-04,-0.537690,-0.537690,-0.537690
2001-01-05,0.897283,0.897283,0.897283
2001-01-06,0.325985,1.749058,0.325985
2001-01-07,0.664227,0.578281,0.122327
2001-01-08,-1.262073,0.142682,-1.262073


This is equivalent to (but faster than) the following

In [192]:
df.apply(lambda x, y: x.where(x > 0, y), y=df['A'])

,A,B,C
2001-01-01,0.199353,0.958447,0.199353
2001-01-02,0.520690,0.520690,0.520690
2001-01-03,-0.813522,0.005863,-0.813522
2001-01-04,-0.537690,-0.537690,-0.537690
2001-01-05,0.897283,0.897283,0.897283
2001-01-06,0.325985,1.749058,0.325985
2001-01-07,0.664227,0.578281,0.122327
2001-01-08,-1.262073,0.142682,-1.262073


### Mask
`mask` is the inverse boolean operation of where

In [193]:
s

4    0
3    1
2    2
1    3
0    4
dtype: int64

In [194]:
s.mask(s >= 3)

4    0.0
3    1.0
2    2.0
1    NaN
0    NaN
dtype: float64

In [195]:
s.where(s >= 3)

4    NaN
3    NaN
2    NaN
1    3.0
0    4.0
dtype: float64

In [196]:
df.mask(df >= 0)

,A,B,C
2001-01-01,NaN,NaN,-0.890055
2001-01-02,NaN,-0.271940,-0.949279
2001-01-03,-0.813522,NaN,-1.872484
2001-01-04,-0.537690,-1.451723,-0.623780
2001-01-05,NaN,-0.642548,-1.014361
2001-01-06,NaN,NaN,-0.988162
2001-01-07,NaN,NaN,NaN
2001-01-08,-1.262073,NaN,-0.515181


---
## 17. The `.query()` Method

DataFrame objects have a `query()` method that allows selection using an expression.

You can get the value of the frame where column b has values between the values of columns a and c. For example:

In [197]:
n = 10
df = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))

In [198]:
df

,a,b,c
0,0.016109,0.553706,0.925779
1,0.275667,0.910336,0.442246
2,0.073279,0.007959,0.236087
3,0.365854,0.473639,0.423130
4,0.869080,0.149310,0.691329
5,0.135838,0.932030,0.761871
6,0.463061,0.902372,0.295278
7,0.386204,0.703819,0.013177
8,0.660749,0.499923,0.635869
9,0.574452,0.345857,0.200945


In [199]:
# pure python
df[(df.a < df.b) & (df.b < df.c)]

,a,b,c
0,0.016109,0.553706,0.925779


In [200]:
#query()
df.query('(a < b) & (b < c)')

,a,b,c
0,0.016109,0.553706,0.925779


Do the same thing but fall back on a named index if there is no column with the name a.

In [210]:
df = pd.DataFrame(np.random.randint(n / 2, size=(n, 2)), columns=list('bc'))

In [211]:
df.index.name = 'a'

In [212]:
df

,b,c
a,,
0,3,4
1,4,1
2,1,0
3,1,2
4,3,1
5,2,0
6,2,3
7,3,4
8,3,3


In [213]:
df.query('a < b and b < c')

,b,c
a,,
0,3,4


If instead you don’t want to or cannot name your index, you can use the name index in your query expression:

In [217]:
df = pd.DataFrame(np.random.randint(n, size=(n,2)), columns=list('bc'))

In [218]:
df

,b,c
0,9,2
1,4,2
2,3,7
3,6,7
4,0,1
5,1,3
6,0,1
7,7,0
8,9,4
9,7,6


In [219]:
df.query('index < b < c')

,b,c
2,3,7
3,6,7


>If the name of your index overlaps with a column name, the column name is given precedence. For example,

In [227]:
df = pd.DataFrame({'a': np.random.randint(5, size=5)})

In [228]:
df.index.name = 'a'

In [229]:
df

,a
a,
0,3
1,0
2,1
3,1
4,2


In [230]:
df.query(  'a > 2'   )

,a
a,
0,3


>You can still use the index in a query expression by using the special identifier ‘index’:

In [231]:
df.query( 'index > 2')

,a
a,
3,1
4,2


> If for some reason you have a column named index, then you can refer to the index as `ilevel_0` as well(The convention is ilevel_0, which means “index level 0” for the 0th level of the index.), but at this point you should consider renaming your columns to something less ambiguous.

In [238]:
df.columns = ['index']
df.index.name = None

In [239]:
df.query( 'ilevel_0 > 2')

,index
3,1
4,2


`ilevel_*` is the name assigned, when the index does not yet have a name. So, this command gives you an `UndefinedVariableError`

### MultiIndex query() Syntax

You can also use the levels of a DataFrame with a `MultiIndex` as if they were columns in the frame:

In [240]:
n = 10
colors = np.random.choice(['red' ,'green'], size=n)
foods = np.random.choice(['eggs', 'ham'], size=n)

In [241]:
colors

array(['green', 'red', 'green', 'green', 'red', 'green', 'green', 'red',
       'green', 'green'], dtype='<U5')

In [242]:
foods

array(['eggs', 'ham', 'ham', 'eggs', 'eggs', 'ham', 'ham', 'eggs', 'ham',
       'ham'], dtype='<U4')

In [243]:
index = pd.MultiIndex.from_arrays([colors, foods], names=['color', 'food'])

In [244]:
index

MultiIndex([('green', 'eggs'),
            (  'red',  'ham'),
            ('green',  'ham'),
            ('green', 'eggs'),
            (  'red', 'eggs'),
            ('green',  'ham'),
            ('green',  'ham'),
            (  'red', 'eggs'),
            ('green',  'ham'),
            ('green',  'ham')],
           names=['color', 'food'])

In [245]:
df = pd.DataFrame(np.random.randn(n,2), index=index)

In [246]:
df

0         1
color food                    
green eggs -0.777342 -0.052012
red   ham  -1.703090 -1.291749
green ham  -2.132367 -0.968937
      eggs -1.476868 -0.215046
red   eggs  0.788112 -1.416917
green ham  -0.251796 -1.296525
      ham  -0.238021 -1.014977
red   eggs  0.196959 -1.090157
green ham   0.700370  0.079486
      ham   1.304402 -1.192497

In [247]:
df.query('color == "red"')

0         1
color food                    
red   ham  -1.703090 -1.291749
      eggs  0.788112 -1.416917
      eggs  0.196959 -1.090157

If the levels of the MultiIndex are unnamed, you can refer to them using special names:  **`ilevel_*`**

In [250]:
df.index.names = [None, None]

In [252]:
df.query('ilevel_0 == "red"')

0         1
red ham  -1.703090 -1.291749
    eggs  0.788112 -1.416917
    eggs  0.196959 -1.090157

### `query()` Use Cases

A use case for `query()` is when you have a collection of DataFrame objects that have a subset of column names (or index levels/names) in common. 

**You can pass the same query to both frames without having to specify which frame you’re interested in querying**

In [253]:
df = pd.DataFrame(np.random.rand(n,3), columns=list('abc'))

In [254]:
df

,a,b,c
0,0.538649,0.798006,0.589358
1,0.764240,0.775806,0.204609
2,0.204202,0.153121,0.311747
3,0.881833,0.762574,0.197333
4,0.418994,0.300483,0.196254
5,0.479947,0.797735,0.916953
6,0.133372,0.050414,0.204028
7,0.362056,0.666239,0.305905
8,0.860663,0.195682,0.145831
9,0.394524,0.429296,0.225537


In [256]:
df2 = pd.DataFrame(np.random.rand(n+2, 3), columns=df.columns)

In [257]:
expr = '0.0 <= a <= c <= 0.5'

In [268]:
map(lambda frame: frame.query(expr), [df, df2])

### `query()` Python vs. pandas Syntax Comparison

full numpy-like syntax

In [269]:
df = pd.DataFrame(  np.random.randint(n,  size=(n,3)),  columns=list('abc'))

In [270]:
df

,a,b,c
0,3,7,3
1,7,6,2
2,4,5,8
3,2,0,4
4,4,4,2
5,2,2,0
6,6,0,5
7,8,9,7
8,0,7,7
9,9,8,2


In [271]:
df.query(   '(a<b<c)')

,a,b,c
2,4,5,8


In [273]:
df[    (df.a < df.b)  &  (df.b  <  df.c)    ]

,a,b,c
2,4,5,8


Slightly nicer by removing the parentheses (by binding making comparison operators bind tighter than & and |).

In [274]:
df.query(   'a<b & b<c')

,a,b,c
2,4,5,8


Or Use English instead of symbols

In [275]:
df.query(   'a < b and b < c')

,a,b,c
2,4,5,8


Pretty close to how you might write it on paper

In [276]:
df.query(  'a < b < c')

,a,b,c
2,4,5,8


### The `in` and `not in` Operators:

query() also supports special use of Python’s in and not in comparison operators, providing a succinct syntax for calling the isin method of a Series or DataFrame.

In [279]:
# get all rows where columns "a" and "b" have overlapping values
df = pd.DataFrame(  {'a': list('aabbccddeeff'), 'b': list('aaaabbbbcccc'),
                     'c': np.random.randint(5, size=12),
                     'd': np.random.randint(9, size=12)})

In [280]:
df

,a,b,c,d
0,a,a,1,6
1,a,a,4,6
2,b,a,4,4
3,b,a,1,5
4,c,b,4,1
5,c,b,1,1
6,d,b,4,0
7,d,b,0,3
8,e,c,0,5
9,e,c,2,3


In [281]:
df.query('a in b')

,a,b,c,d
0,a,a,1,6
1,a,a,4,6
2,b,a,4,4
3,b,a,1,5
4,c,b,4,1
5,c,b,1,1


In [282]:
# Pure Python version
df[df.a.isin(df.b)]

,a,b,c,d
0,a,a,1,6
1,a,a,4,6
2,b,a,4,4
3,b,a,1,5
4,c,b,4,1
5,c,b,1,1


`Not in`

In [283]:
df.query(  'a not in b')

,a,b,c,d
6,d,b,4,0
7,d,b,0,3
8,e,c,0,5
9,e,c,2,3
10,f,c,0,0
11,f,c,2,4


In [284]:
df[~df.a.isin(df.b)]

,a,b,c,d
6,d,b,4,0
7,d,b,0,3
8,e,c,0,5
9,e,c,2,3
10,f,c,0,0
11,f,c,2,4


You can combine this with other expressions for very succint queries:

In [286]:
# rows where cols a and b have overlapping values and col 'cs values are less than col d's

In [287]:
df.query( 'a in b and c < d')

,a,b,c,d
0,a,a,1,6
1,a,a,4,6
3,b,a,1,5


In [289]:
# pure Python
df[df.b.isin(df.a) & (df.c < df.d)]

,a,b,c,d
0,a,a,1,6
1,a,a,4,6
3,b,a,1,5
7,d,b,0,3
8,e,c,0,5
9,e,c,2,3
11,f,c,2,4


> Note that `in` and `not in` are evaluated in **Python**, since numexpr has no equivalent of this operation. However, only the in/not in expression itself is evaluated in vanilla Python. For example, in the expression

(b + c + d) is evaluated by numexpr and then the in operation is evaluated in plain Python. In general, any operations that can be evaluated using numexpr will be.

### Special use of the `==` operator with `list` objects

Comparing a list of values to a column using ==/!= works similarly to in/not in.

In [311]:
df

,a,b,c,d
0,a,a,1,6
1,a,a,4,6
2,b,a,4,4
3,b,a,1,5
4,c,b,4,1
5,c,b,1,1
6,d,b,4,0
7,d,b,0,3
8,e,c,0,5
9,e,c,2,3


In [299]:
df.query('b == ["a", "b", "c"]')

,a,b,c,d
0,a,a,1,6
1,a,a,4,6
2,b,a,4,4
3,b,a,1,5
4,c,b,4,1
5,c,b,1,1
6,d,b,4,0
7,d,b,0,3
8,e,c,0,5
9,e,c,2,3


In [301]:
# == special use... similar to in
df.query(  'b in ["a", "b", "c"]')

,a,b,c,d
0,a,a,1,6
1,a,a,4,6
2,b,a,4,4
3,b,a,1,5
4,c,b,4,1
5,c,b,1,1
6,d,b,4,0
7,d,b,0,3
8,e,c,0,5
9,e,c,2,3


In [303]:
# pure python
df[df.b.isin(['a', 'b', 'c'])]

,a,b,c,d
0,a,a,1,6
1,a,a,4,6
2,b,a,4,4
3,b,a,1,5
4,c,b,4,1
5,c,b,1,1
6,d,b,4,0
7,d,b,0,3
8,e,c,0,5
9,e,c,2,3


In [304]:
df.query(  'c == [1,2]')

,a,b,c,d
0,a,a,1,6
3,b,a,1,5
5,c,b,1,1
9,e,c,2,3
11,f,c,2,4


In [305]:
# == special use similar to in
df.query(  'c in [1,2]')

,a,b,c,d
0,a,a,1,6
3,b,a,1,5
5,c,b,1,1
9,e,c,2,3
11,f,c,2,4


In [306]:
# pure python
df[df.c.isin( [1, 2] )]

,a,b,c,d
0,a,a,1,6
3,b,a,1,5
5,c,b,1,1
9,e,c,2,3
11,f,c,2,4


In [307]:
df.query(' c != [1,2]')

,a,b,c,d
1,a,a,4,6
2,b,a,4,4
4,c,b,4,1
6,d,b,4,0
7,d,b,0,3
8,e,c,0,5
10,f,c,0,0


In [308]:
# != special use similar to not in 
df.query(  'c not in [1, 2]')

,a,b,c,d
1,a,a,4,6
2,b,a,4,4
4,c,b,4,1
6,d,b,4,0
7,d,b,0,3
8,e,c,0,5
10,f,c,0,0


In [309]:
# pure python
df[~df.c.isin([1,2])]

,a,b,c,d
1,a,a,4,6
2,b,a,4,4
4,c,b,4,1
6,d,b,4,0
7,d,b,0,3
8,e,c,0,5
10,f,c,0,0


In [310]:
df.query(  '[1,2] in c')

,a,b,c,d
0,a,a,1,6
3,b,a,1,5
5,c,b,1,1
9,e,c,2,3
11,f,c,2,4


In [312]:
df.query(  '[1,2] not in c')

,a,b,c,d
1,a,a,4,6
2,b,a,4,4
4,c,b,4,1
6,d,b,4,0
7,d,b,0,3
8,e,c,0,5
10,f,c,0,0


In [314]:
#pure python
df[~df.c.isin([1,2])]

,a,b,c,d
1,a,a,4,6
2,b,a,4,4
4,c,b,4,1
6,d,b,4,0
7,d,b,0,3
8,e,c,0,5
10,f,c,0,0


### Boolean operator
You can negate boolean expressions with the word `not` or the `~` operator

In [315]:
df = pd.DataFrame(  np.random.rand(n, 3), columns=list('abc'))

In [317]:
df['bools'] = np.random.rand(len(df)) > 0.5

In [318]:
df

,a,b,c,bools
0,0.328274,0.535650,0.663603,False
1,0.229007,0.144537,0.134162,False
2,0.561783,0.090495,0.363449,False
3,0.287311,0.811148,0.844135,True
4,0.943069,0.311981,0.622348,False
5,0.778663,0.443212,0.402092,False
6,0.803842,0.486978,0.442012,True
7,0.212014,0.339838,0.032107,False
8,0.885084,0.558747,0.032903,False
9,0.925205,0.149029,0.692934,True


In [319]:
df.query('bools')

,a,b,c,bools
3,0.287311,0.811148,0.844135,True
6,0.803842,0.486978,0.442012,True
9,0.925205,0.149029,0.692934,True


In [320]:
df.query('not bools')

,a,b,c,bools
0,0.328274,0.535650,0.663603,False
1,0.229007,0.144537,0.134162,False
2,0.561783,0.090495,0.363449,False
4,0.943069,0.311981,0.622348,False
5,0.778663,0.443212,0.402092,False
7,0.212014,0.339838,0.032107,False
8,0.885084,0.558747,0.032903,False


In [321]:
df.query('~bools')

,a,b,c,bools
0,0.328274,0.535650,0.663603,False
1,0.229007,0.144537,0.134162,False
2,0.561783,0.090495,0.363449,False
4,0.943069,0.311981,0.622348,False
5,0.778663,0.443212,0.402092,False
7,0.212014,0.339838,0.032107,False
8,0.885084,0.558747,0.032903,False


In [322]:
df.query('not bools')  == df[~df.bools]

,a,b,c,bools
0,True,True,True,True
1,True,True,True,True
2,True,True,True,True
4,True,True,True,True
5,True,True,True,True
7,True,True,True,True
8,True,True,True,True


Of course, expressions can be arbitrarily complex too:

In [333]:
my_expr = [df.at[0,'a'], df.at[1,'a']]

In [335]:
df.query('a in @my_expr')  # use @ to call variable in environment

,a,b,c,bools
0,0.328274,0.535650,0.663603,False
1,0.229007,0.144537,0.134162,False


In [324]:
# short query syntax
shorter = df.query(  'a < b < c and (not bools) or bools > 2')

In [325]:
# equivalent in pure Python
longer = df[  (df.a < df.b) & (df.b < df.c) & (~df.bools) | (df.bools > 2)]

In [327]:
shorter

,a,b,c,bools
0,0.328274,0.53565,0.663603,False


In [328]:
longer

,a,b,c,bools
0,0.328274,0.53565,0.663603,False


In [329]:
shorter == longer

,a,b,c,bools
0,True,True,True,True


### Performance of query()

DataFrame.query() using `numexpr` is slightly faster than pure python for large frames

Note You will only see the performance benefits of using the numexpr engine with DataFrame.query() if your frame has more than approximately 200,000 rows.

20만줄 이상 데이터에 대해서 유리함

---
## 18. Mark / Drop Duplicate data : `.duplicated()` ; `.drop_duplicates()`

If you want to identify and remove **duplicate rows** in a DataFrame, there are two methods that will help: `.duplicated()` and `.drop_duplicates()`. Each takes as an argument the columns to use to identify duplicated rows.

- `duplicated` **MARKING** returns a boolean vector whose length is the number of rows, and which indicates whether a row is duplicated.
- `drop_duplicates` **Drop** removes duplicate rows.


By default, **the first observed row** of a duplicate set is considered unique, but each method has a keep parameter to **specify targets to be kept**.

- `keep='first'` (default): mark / drop duplicates except for the first occurrence.
- `keep='last'`: mark / drop duplicates except for the last occurrence.
- `keep=False`: mark / drop all duplicates.

In [336]:
df2 = pd.DataFrame(   {'a': ['one', 'one', 'two', 'two', 'two', 'three', 'four'],
                       'b': ['x', 'y', 'x', 'y', 'x', 'x', 'x'], 
                       'c': np.random.randn(7)})

In [337]:
df2

,a,b,c
0,one,x,-0.368203
1,one,y,1.186379
2,two,x,-1.311168
3,two,y,0.108683
4,two,x,0.921771
5,three,x,0.596329
6,four,x,-1.131808


In [338]:
df2.duplicated('a')

0    False
1     True
2    False
3     True
4     True
5    False
6    False
dtype: bool

In [339]:
df2.duplicated('a', keep='last')

0     True
1    False
2     True
3     True
4    False
5    False
6    False
dtype: bool

In [341]:
df2.duplicated('a', keep=False)

0     True
1     True
2     True
3     True
4     True
5    False
6    False
dtype: bool

In [342]:
df2.drop_duplicates('a')

,a,b,c
0,one,x,-0.368203
2,two,x,-1.311168
5,three,x,0.596329
6,four,x,-1.131808


In [343]:
df2.drop_duplicates('a', keep='last')

,a,b,c
1,one,y,1.186379
4,two,x,0.921771
5,three,x,0.596329
6,four,x,-1.131808


In [344]:
df2.drop_duplicates('a', keep=False)

,a,b,c
5,three,x,0.596329
6,four,x,-1.131808


Also, you can pass a list of columns to identify duplications.

In [345]:
df2.duplicated(  ['a', 'b']  )

0    False
1    False
2    False
3    False
4     True
5    False
6    False
dtype: bool

In [346]:
df2.drop_duplicates(  ['a', 'b']  )

,a,b,c
0,one,x,-0.368203
1,one,y,1.186379
2,two,x,-1.311168
3,two,y,0.108683
5,three,x,0.596329
6,four,x,-1.131808


To drop duplicates **by index value**, use `Index.duplicated` then perform slicing. The same set of options are available for the `keep=*` parameter.

In [351]:
df3 = pd.DataFrame(    {'a': np.arange(6),
                        'b': np.random.randn(6)},
                   index=['a', 'a', 'b', 'c', 'b', 'a'])

In [352]:
df3

,a,b
a,0,0.007926
a,1,1.267309
b,2,0.096723
c,3,-0.326280
b,4,1.018294
a,5,0.009316


In [353]:
df3.index.duplicated()

array([False,  True, False, False,  True,  True])

In [354]:
df3[~df3.index.duplicated()]

,a,b
a,0,0.007926
b,2,0.096723
c,3,-0.326280


In [355]:
df3[~df3.index.duplicated(keep='last')]

,a,b
c,3,-0.326280
b,4,1.018294
a,5,0.009316


In [356]:
df3[ ~ df3.index.duplicated(keep=False)]

,a,b
c,3,-0.32628


---
## 19. Dictionary-like `.get()` method

Each of Series or DataFrame have a `s/df.get()` method which can return a default value.

In [357]:
s = pd.Series([1,2,3],  index=['a', 'b', 'c'])

In [358]:
s.get('a')

1

In [359]:
s.get('x', default=-1)

-1

In [361]:
s.get('b', default=-1)

2

In [363]:
df3.get('a', default='b')

a    0
a    1
b    2
c    3
b    4
a    5
Name: a, dtype: int32

In [366]:
df3.get('x', 
         default=df3.get('b'))

a    0.007926
a    1.267309
b    0.096723
c   -0.326280
b    1.018294
a    0.009316
Name: b, dtype: float64

---
## 20. the `.lookup()` method

Sometimes you want to extract a set of values given a sequence of row labels and column labels, and the lookup method allows for this and returns a NumPy array. For instance:

In [376]:
dflookup = pd.DataFrame(  np.random.randint(0, high=100, size=(20, 4)), 
                       columns= list('ABCD'))

In [379]:
dflookup.lookup(  list(  range(0, 10, 2)),   ['B', 'C', 'A', 'B', 'D'])  #(0,'B'), (2,'C'), (4,'A'), (6, 'B'), (8,'D') return

array([55, 27, 91, 22, 61])

In [385]:
[ dflookup.at[row, col] 
  for row, col in zip(range(0, 10, 2), ['B', 'C', 'A', 'B', 'D'])   ]

[55, 27, 91, 22, 61]

In [386]:
[  dflookup.at[row, col] 
            for row, col in zip(  range(0, 10, 2),  ['B', 'C', 'A', 'B', 'D'])]

[55, 27, 91, 22, 61]

In [413]:
# Random generator
[   dflookup.iat[row, col] 
 for row, col in zip( np.random.randint(0, high=19, size=5), 
                      np.random.randint(0, high=3, size=5))    ]

[48, 3, 22, 52, 22]

---
## 21. Index object

The pandas Index class and its subclasses can be viewed as implementing an ordered multiset. Duplicates are allowed. However, if you try to convert an Index object with duplicate entries into a set, an exception will be raised.

Index also provides the infrastructure necessary for `lookups`, `data alignment`, and `reindexing`. The easiest way to create an Index directly is to pass a list or other sequence to Index:

In [430]:
index = pd.Index(['e', 'd', 'a', 'b'])

In [431]:
index

Index(['e', 'd', 'a', 'b'], dtype='object')

In [432]:
'd' in index

True

You can also pass a `name=` to be stored in index:

In [433]:
index = pd.Index(['e', 'd', 'a', 'b'], name='my_index')

In [434]:
index

Index(['e', 'd', 'a', 'b'], dtype='object', name='my_index')

The name, if set, will be shown in the console display:

In [436]:
index = pd.Index(list(range(5)), name='rows')

In [437]:
columns = pd.Index(list('ABC'), name='cols')

In [439]:
df = pd.DataFrame(np.random.randn(len(index), len(columns)),
                  index=index, columns=columns)

In [440]:
df

cols,A,B,C
rows,,,
0,-0.732905,-0.770918,1.293215
1,0.130131,0.033955,0.379099
2,-2.065102,-1.470682,0.724824
3,0.923886,0.208498,-0.924932
4,0.156099,1.106935,0.484831


In [442]:
df['A'] # 0 위에 'rows' 라는 이름이 표시됨

rows
0   -0.732905
1    0.130131
2   -2.065102
3    0.923886
4    0.156099
Name: A, dtype: float64

### Setting metadata!

Indexes are “mostly immutable”(change attributes), but it is possible to set and change their metadata, like the index `name=` (or, for MultiIndex, `levels` and `codes`).

You can use the `rename`, `set_names`, `set_levels`, and `set_codes` to set these attributes directly. They default to returning a copy; however, you can specify inplace=True to have the data change in place.

See [Advanced Indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced) for usage of MultiIndexes.

In [443]:
ind = pd.Index([1, 2, 3])

In [444]:
ind.rename("apple")

Int64Index([1, 2, 3], dtype='int64', name='apple')

In [445]:
ind

Int64Index([1, 2, 3], dtype='int64')

In [446]:
ind.set_names("apple", inplace=True)

In [447]:
ind

Int64Index([1, 2, 3], dtype='int64', name='apple')

In [448]:
ind.name = "bob"

In [449]:
ind

Int64Index([1, 2, 3], dtype='int64', name='bob')

In [454]:
ind[2] = 99

TypeError: Index does not support mutable operations

`set_names`, `set_levels` and `set_codes` also take an optional `level=` argument

In [455]:
index = pd.MultiIndex.from_product(  [range(3),   ['one', 'two']  ], names=['first', 'second'] )

In [456]:
index

MultiIndex([(0, 'one'),
            (0, 'two'),
            (1, 'one'),
            (1, 'two'),
            (2, 'one'),
            (2, 'two')],
           names=['first', 'second'])

In [460]:
index.levels[1]

Index(['one', 'two'], dtype='object', name='second')

In [461]:
index.set_levels(["a", "b"], level=1)

MultiIndex([(0, 'a'),
            (0, 'b'),
            (1, 'a'),
            (1, 'b'),
            (2, 'a'),
            (2, 'b')],
           names=['first', 'second'])

### Set operations on Index objects: .union, .intersection, .difference

The two main operations are union (|) and intersection (&). These can be directly called as instance methods or used via overloaded operators. Difference is provided via the .difference() method.

In [482]:
a = pd.Index(list('abc'))

In [483]:
b = pd.Index(list('cde'))

In [484]:
a | b

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

In [486]:
a.union(b)

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

In [485]:
a & b

Index(['c'], dtype='object')

In [487]:
a.intersection(b)

Index(['c'], dtype='object')

In [488]:
a.difference(b)

Index(['a', 'b'], dtype='object')

In [489]:
b.difference(a)

Index(['d', 'e'], dtype='object')

Also available is the `.symmetric_difference` (^) operation, which returns elements that appear in either idx1 or idx2, but not in both. This is equivalent to the Index created by idx1.difference(idx2).union(idx2.difference(idx1)), with duplicates dropped.

In [490]:
idx1 = pd.Index([1,2,3,4])
idx2 = pd.Index([2,3,4,5])

In [491]:
idx1.symmetric_difference(idx2)

Int64Index([1, 5], dtype='int64')

In [492]:
idx1.difference(idx2) | idx2.difference(idx1)

Int64Index([1, 5], dtype='int64')

> Note: The resulting index from a set operation will be **sorted** in ascending order.

When performing Index.union() between indexes with different dtypes, the indexes must be cast to a common dtype. Typically, though not always, this is object dtype. The exception is when performing a union between integer and float data. In this case, the integer values are converted to float

In [493]:
idx1 = pd.Index([0, 1, 2])
idx2 = pd.Index([0.5, 1.0, 1.5])

In [494]:
idx1 | idx2

Float64Index([0.0, 0.5, 1.0, 1.5, 2.0], dtype='float64')

In [495]:
idx1 & idx2

Float64Index([1.0], dtype='float64')

In [496]:
idx1 ^ idx2

Float64Index([0.0, 0.5, 1.5, 2.0], dtype='float64')

### Missing values

> **IMPORTANT** Even though Index can hold missing values (NaN), it should be avoided if you do not want any unexpected results. For example, some operations exclude missing values implicitly.

Index.fillna fills missing values with specified scalar value.

In [497]:
idx1 = pd.Index([1, np.nan, 3, 4])

In [498]:
idx1

Float64Index([1.0, nan, 3.0, 4.0], dtype='float64')

In [499]:
idx1.fillna(2)

Float64Index([1.0, 2.0, 3.0, 4.0], dtype='float64')

In [500]:
idx1

Float64Index([1.0, nan, 3.0, 4.0], dtype='float64')

In [501]:
idx2 = pd.DatetimeIndex([pd.Timestamp('20110101'),
                         pd.NaT,
                         pd.Timestamp('20110103')])

In [502]:
idx2

DatetimeIndex(['2011-01-01', 'NaT', '2011-01-03'], dtype='datetime64[ns]', freq=None)

In [503]:
idx2.fillna(pd.Timestamp('20110102'))

DatetimeIndex(['2011-01-01', '2011-01-02', '2011-01-03'], dtype='datetime64[ns]', freq=None)

In [506]:
idx2.fillna(pd.Timestamp('20110102'), inplace=True) ## ERROR: FROZEN !!!

TypeError: fillna() got an unexpected keyword argument 'inplace'

---
## 22. Set/reset Index

Occasionally you will load or create a data set into a DataFrame and want to add an index after you’ve already done so. There are a couple of different ways.

### Set an index to DataFrame

DataFrame has a `set_index()` method which takes a column name (for a regular Index) or a list of column names (for a MultiIndex). To create a new, re-indexed DataFrame:

In [560]:
data = pd.DataFrame({'a': ['bar', 'bar', 'foo', 'foo'], 
                     'b': ['one', 'two', 'one', 'two'], 
                     'c': ['z', 'y', 'x', 'w'],
                     'd': [1.0, 2.0, 3.0, 4.0]         })

In [561]:
data

,a,b,c,d
0,bar,one,z,1.0
1,bar,two,y,2.0
2,foo,one,x,3.0
3,foo,two,w,4.0


In [562]:
indexed1 = data.set_index('c')

In [563]:
indexed1

,a,b,d
c,,,
z,bar,one,1.0
y,bar,two,2.0
x,foo,one,3.0
w,foo,two,4.0


In [564]:
indexed2 = data.set_index(['a', 'b'])

In [565]:
indexed2

c    d
a   b          
bar one  z  1.0
    two  y  2.0
foo one  x  3.0
    two  w  4.0

The `append=` keyword option allow you to keep the existing index and append the given columns to a MultiIndex:

In [566]:
frame = data.set_index('c', drop=False)

In [567]:
frame

,a,b,c,d
c,,,,
z,bar,one,z,1.0
y,bar,two,y,2.0
x,foo,one,x,3.0
w,foo,two,w,4.0


In [568]:
frame.set_index(['a', 'b'], append=True)

,,,c,d
c,a,b,,
z,bar,one,z,1.0
y,bar,two,y,2.0
x,foo,one,x,3.0
w,foo,two,w,4.0


Other options in set_index allow you not drop the index columns or to add the index in-place (without creating a new object):

In [569]:
data.set_axis(data.a, axis=0, inplace=True)

In [570]:
data.reset_index(drop=True)

,a,b,c,d
0,bar,one,z,1.0
1,bar,two,y,2.0
2,foo,one,x,3.0
3,foo,two,w,4.0


### Reset the index

As a convenience, there is a new function on DataFrame called `reset_index()` which **transfers the index values** `into` the DataFrame’s columns and sets a simple integer index. This is the inverse operation of set_index().

In [571]:
data.set_index(['a', 'b'], drop=True, inplace=True)

In [572]:
data

c    d
a   b          
bar one  z  1.0
    two  y  2.0
foo one  x  3.0
    two  w  4.0

In [573]:
data.reset_index()

,a,b,c,d
0,bar,one,z,1.0
1,bar,two,y,2.0
2,foo,one,x,3.0
3,foo,two,w,4.0


The output is more similar to a SQL table or a record array. The names for the columns derived from the index are the ones stored in the names attribute.

You can use the level keyword to remove only a portion of the index:

In [574]:
frame

,a,b,c,d
c,,,,
z,bar,one,z,1.0
y,bar,two,y,2.0
x,foo,one,x,3.0
w,foo,two,w,4.0


In [575]:
frame.set_index(['a','b'], inplace=True, append=True)

In [576]:
frame

,,,c,d
c,a,b,,
z,bar,one,z,1.0
y,bar,two,y,2.0
x,foo,one,x,3.0
w,foo,two,w,4.0


In [583]:
frame.reset_index(level=1, inplace=True)

In [584]:
frame.reset_index(level=1)

,b,a,c,d
c,,,,
z,one,bar,z,1.0
y,two,bar,y,2.0
x,one,foo,x,3.0
w,two,foo,w,4.0


reset_index takes an optional parameter drop which if true simply discards the index, instead of putting index values in the DataFrame’s columns.

### Adding an ad-hoc index

if you create an index yourself, you can just assign it 

In [585]:
data

c    d
a   b          
bar one  z  1.0
    two  y  2.0
foo one  x  3.0
    two  w  4.0

In [589]:
index[:4]

MultiIndex([(0, 'one'),
            (0, 'two'),
            (1, 'one'),
            (1, 'two')],
           names=['first', 'second'])

In [590]:
data.index = index[:4]

In [591]:
data

c    d
first second        
0     one     z  1.0
      two     y  2.0
1     one     x  3.0
      two     w  4.0

---
## 23. Returning a view versus a copy

When setting values in a pandas object, care must be taken to avoid what is called `chained indexing`. Here is an example

In [203]:
dfmi = pd.DataFrame([list('abcd'),
                     list('efgh'),
                     list('ijkl'),
                     list('mnop')],
                   columns=pd.MultiIndex.from_product([['one','two'],
                                                       ['first','second']]))

In [204]:
dfmi

one          two       
  first second first second
0     a      b     c      d
1     e      f     g      h
2     i      j     k      l
3     m      n     o      p

Compare these two access methods:

In [206]:
dfmi['one']['second']

0    b
1    f
2    j
3    n
Name: second, dtype: object

In [207]:
dfmi.loc[:, ('one', 'second')]

0    b
1    f
2    j
3    n
Name: (one, second), dtype: object

These both yield the same results, so which should you use? It is instructive to understand the order of operations on these and why method 2 (.loc) is much preferred over method 1 (chained []).

`dfmi['one']` selects the first level of the columns and returns a DataFrame that is singly-indexed. Then another Python operation dfmi_with_one['second'] selects the series indexed by 'second'. This is indicated by the variable dfmi_with_one because pandas sees these operations as separate events. e.g. separate calls to __getitem__, so it has to treat them as linear operations, they happen one after another.

Contrast this to df.loc[:,('one','second')] which passes a nested tuple of (slice(None),('one','second')) to a single call to __getitem__. This allows pandas to deal with this as a single entity. Furthermore this order of operations can be significantly faster, and allows one to index both axes if so desired.

In [ ]:
dfmi.loc[:, ('one', 'second')] = 